In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import dask.dataframe as dd

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## Download the data 

In [2]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/" # This notebook should be ran mannually

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

for file in files:
    if file["name"] == "data.zip":
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

In [3]:
output_directory = "../data/" # This notebook should be ran mannually

## Combine the data

In [5]:
%%time
%%memit
# shows time that dask take to merge

combined_data = dd.from_pandas(pd.DataFrame({'time': [], "lat_min": [], "lat_max": [], "lon_min": [], "lon_max": [], "rain (mm/day)": [], "model": []}), chunksize=500)

for filename in os.listdir(output_directory):
    if filename[-4: ] == ".csv":
        model = filename.partition('_daily_rainfall_NSW')[0]
        ddf = dd.read_csv(output_directory + filename, assume_missing=True)
        if len(ddf.columns) == 2:
            ddf['lat_min'] = None
            ddf['lat_max'] = None
            ddf['lon_min'] = None
            ddf['lon_max'] = None
            ddf = ddf[['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']]
        ddf["model"] = model
        combined_data = dd.concat([combined_data, ddf], axis=0)

combined_data.to_csv(output_directory + "combined_data.csv")

peak memory: 3155.68 MiB, increment: 2984.49 MiB
CPU times: user 13min 15s, sys: 59.5 s, total: 14min 15s
Wall time: 12min 10s


## EDA in python

## EDA in R